0 - Sequence to Sequence Learning with Neural Networks
-------------
在本系列中，我们将使用PyTorch和TorchText构建一个机器学习模型，从一次序列到另一个序列。 这将在德语到英语翻译中完成，但模型可以应用于涉及从一个序列到另一个序列的任何问题，例如摘要。

在第一本笔记本中，我们将通过实现从神经网络的序列到序列学习的模型，开始简单地理解一般概念。

Introduction
----------------------
最常见的序列到序列（seq2seq）模型是编码器 - 解码器模型，其（通常）使用递归神经网络（RNN）将源（输入）句子编码成单个矢量。 在这个笔记本中，我们将这个单个向量称为上下文向量。 您可以将上下文向量视为源语言中输入句子的抽象表示。 然后，该矢量由第二RNN解码，该第二RNN通过一次生成一个字来学习输出目标（输出）句子。

<img src='tmp/img/9.png'>

输入/源句“guten morgen”一次一个字地输入编码器（绿色）。 我们还将“序列开始”（`<sos>`）和“序列结束”（`<eos>`）标记分别附加到句子的开头和结尾。 在每个时间步，编码器RNN的输入都是当前字$ x_t $，以及来自前一时间步的隐藏状态$ h_ {t-1} $，并且编码器RNN输出 新的隐藏状态$ h_t $。 您可以将RNN视为这两个输入的函数：

$$h_t = \text{RNN}(x_t, h_{t-1})$$

我们这里通常使用术语RNN，它可以是任何循环架构，例如LSTM（长短期存储器）或GRU（门控循环单元）。

在这里，我们有$ x_1 = \ text {＆lt; sos＆gt;}，x_2 = \ text {guten} $等。初始隐藏状态$ h_0 $通常初始化为零或学习参数。

一旦最后一个单词$ x_T $被传递到RNN，我们使用最终隐藏状态$ h_T $作为上下文向量，即$ h_T = z $。

现在我们有了上下文向量，我们可以开始解码它来得到输出/目标句子，“早上好”。同样，我们将序列标记的开头和结尾附加到目标句子。在每个时间步，解码器RNN（蓝色）的输入是当前字，$ y_t $，以及来自前一时间步的隐藏状态，$ s_ {t-1} $，其中初始解码器隐藏状态$ s_0 $是上下文向量，$ s_0 = z = h_T $，即初始解码器隐藏状态是最终编码器隐藏状态。

在每个时间步骤，我们还使用$ s_t $来预测（通过传递线性层，以紫色显示）我们认为序列中的下一个单词$ \ {y} _t $。我们总是使用<`sos`>兑换$ y_1 $，但是对于$ y _ {＆gt; 1} $，我们有时会使用序列中的实际下一个词$ y_t $，有时会使用最后预测的单词$ \ hat {y} _ {T-1} $。这被称为教师强制，您可以在这里阅读更多内容。

在训练/测试我们的模型时，我们总是知道目标句子中有多少单词，所以一旦我们达到那么多，我们就会停止生成单词。在推理（即现实世界使用）期间，通常保持生成单词直到模型输出<eos>标记或者在生成一定量的单词之后。

一旦我们得到了预测的目标句子，$ \ hat {Y} = \ {\ hat {y} _1，\ hat {y} _2，...，\ hat {y} _T \} $，我们将它与我们的比较实际目标句子，$ Y = \ {y_1，y_2，...，y_T \} $，以计算我们的损失。然后我们使用此损失来更新模型中的所有参数。

准备数据
------------
我们将在PyTorch中编写模型并使用TorchText帮助我们完成所需的所有预处理。 我们还将使用spaCy来协助数据的标记化。

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import spacy

import random
import math
import os

D:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
D:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


In [4]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

接下来，我们将创建tokenizer。 令牌化器用于将包含句子的字符串转换为构成该字符串的各个令牌的列表，例如， “早上好！” 成为[“早上”，“好”，“！”]。 我们将从现在开始讨论句子是一系列令牌，而不是说它们是一系列单词。 有什么不同？ 好吧，“好”和“早晨”都是单词和代币，但是“！” 是一种象征，而不是一个词。

spaCy有每种语言的模型（德语为“de”，英语为“en”）需要加载，因此我们可以访问每个模型的标记器。

**注意：必须首先使用命令行中的以下命令下载模型：**

* python -m spacy download en
* python -m spacy download de

In [24]:
SRC = Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<eos>', lower=True)
TRG = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>', lower=True)

In [25]:
train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(SRC, TRG))

downloading training.tar.gz


.data\multi30k\training.tar.gz: 100%|██████████████████████████████████████████████| 1.21M/1.21M [00:05<00:00, 224kB/s]


downloading validation.tar.gz


.data\multi30k\validation.tar.gz: 100%|████████████████████████████████████████████| 46.3k/46.3k [00:00<00:00, 114kB/s]


downloading mmt_task1_test2016.tar.gz


.data\multi30k\mmt_task1_test2016.tar.gz: 100%|███████████████████████████████████| 66.2k/66.2k [00:00<00:00, 81.7kB/s]


In [26]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [27]:
print(vars(train_data.examples[0]))

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'm', 'i', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


In [28]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 7875
Unique tokens in target (en) vocabulary: 5893


准备数据的最后一步是创建迭代器。这些可以被迭代以返回一批具有src属性的数据（包含一批数字源句子的PyTorch张量）和trg属性（包含一批数字化目标句子的PyTorch张量）。数字化只是一种奇特的方式，即使用词汇表将它们从一系列可读令牌转换为一系列相应的索引。

我们还需要定义一个`torch.device`。这用于告诉TorchText将张量放在GPU上。我们使用`torch.cuda.is_available（）`函数，如果在我们的计算机上检测到GPU，它将返回True。我们将此设备传递给迭代器。

当我们使用迭代器获得一批示例时，我们需要确保所有源句子都填充到相同的长度，与目标句子相同。幸运的是，TorchText迭代器为我们处理这个！

我们使用`BucketIterator`而不是标准`Iterator`，因为它以这样的方式创建批处理，以便最小化源句和目标句子中的填充量。

In [29]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), batch_size=BATCH_SIZE, device=device)

Building the Seq2Seq Model
------------------------------
我们将分三个部分构建我们的模型。 编码器，解码器和封装编码器和解码器的seq2seq模型将提供与每个编码器和解码器接口的方式。

Encoder
-------------------

首先，编码器，2层LSTM。 我们正在实施的论文使用了4层LSTM，但为了培训时间，我们将其减少到2层。 多层RNN的概念很容易从2层扩展到4层。

对于多层RNN，输入句子进入RNN的第一（底部）层，并且该层输出的隐藏状态用作下一层RNN的输入。 这意味着我们还需要每层初始隐藏状态$ h_0 $，我们还将输出每层的上下文向量$ z $。 两个上下文向量都是不同的，理想情况下我们应该用$ z_l $来表示上下文向量，其中$ l $表示上下文向量来自哪个层，但是为了保持图像清晰，我省略了这个 到处添加图层下标。

没有详细介绍LSTM（如果你想了解更多关于它们的信息，请参阅这篇博客文章），我们需要知道的是，它们是一种RNN，而不仅仅是处于隐藏状态并返回一个新的 隐藏状态，也接收并返回一个单元状态，$ c $。

$$h_t = \text{RNN}(x_t, h_{t-1})$$$$(h_t, c_t) = \text{LSTM}(x_t, (h_{t-1}, c_{t-1}))$$

您可以将$ c $视为另一种隐藏状态。 这意味着我们的上下文向量将是最终隐藏状态和最终单元状态，即$ z =（h_T，c_T）$。

所以我们的编码器看起来像这样：

<img src='tmp/img/10.png'>

我们通过创建一个Encoder模块在代码中创建它，这需要我们从torch.nn.Module继承并使用`super().__ init __()` 作为一些样板代码。 编码器采用以下参数：

* `input_dim`是将输入到编码器的单热矢量的大小/维数。这等于输入（源）词汇量大小。
* `emb_dim`是嵌入层的维度。该层将单热矢量转换为具有emb_dim维度的密集矢量。
* `hid_dim`是隐藏状态和单元状态的维度。
* `n_layers`是RNN中的层数。
* `dropout`是要使用的丢失量。这是一个正则化参数，用于防止过度拟合。有关丢失的详细信息，请查看此内容。


使用nn.Embedding，带有nn.LSTM的LSTM和带有nn.Dropout的dropout层创建嵌入层。有关这些内容的更多信息，请查看PyTorch文档。

需要注意的一点是，LSTM的dropout参数是在多层RNN的层之间应用多少丢失，即在层$ L $输出的隐藏状态和用于输入的相同隐藏状态之间。图层$ L + 1 $。

在前向方法中，我们传入源语句，将其转换为密集向量，然后应用dropout。然后将这些嵌入传递到RNN。当我们将整个序列传递给RNN时，它会自动为整个序列重复计算隐藏状态！您可能会注意到我们没有将初始隐藏或单元状态传递给RNN。这是因为，如文档中所述，如果没有将隐藏/单元状态传递给RNN，它将自动创建初始隐藏/单元状态作为全零的张量。

RNN返回：输出（每个时间步/令牌的顶层隐藏状态），隐藏（每个层的最终隐藏状态，$ h_T $，堆叠在彼此之上）和单元格（最终单元状态为每一层，$ c_T $，叠加在彼此之上）。我们没有将初始隐藏或单元状态传递给RNN，PyTorch将其解释为将它们初始化为全零的张量。

由于我们只需要最终的隐藏和单元格状态（以制作我们的上下文向量），因此只返回隐藏和单元格。

每个张量的大小在代码中留作注释。在此实现中，n_directions将始终为1，但是双向RNN（在教程3中介绍）将具有n_directions为2。

In [30]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [sent len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [sent len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

Decoder
-----------------

接下来，我们将构建我们的解码器，它也将是一个2层（论文中为4个）LSTM。

<img src='tmp/img/11.png'>


解码器类只执行一个解码步骤。它将从前一个时间步骤$（s_ {t-1}，c_ {t-1}）$接收隐藏和单元格状态，并通过LSTM以当前标记$ y_t $提供它，以生成一个新的隐藏和单元格状态，$（s_t，c_t）$。然后我们通过线性层传递隐藏状态$ s_t $，以预测目标（输出）序列中的下一个标记应该是什么，$ \widehat {y} _t $。

参数和初始化类似于Encoder类，除了我们现在有一个output_dim，它是将输入到解码器的单热矢量的大小。这些等于输出/目标的词汇量大小。还增加了线性层，用于从隐藏状态进行预测，即：

$$ \widehat{y} _ {t + 1} = f（s_t）$$
在forward方法中，我们接受一批输入令牌，先前的隐藏状态和先前的单元状态。我们解压缩输入标记以添加1的句子长度维度。然后，类似于编码器，我们通过嵌入层并应用dropout。然后将这批嵌入式令牌传递到具有先前隐藏和单元状态的RNN。这产生一个输出（来自RNN顶层的隐藏状态），一个新的隐藏状态（每个层一个，堆叠在彼此之上）和一个新的单元状态（每层也有一个，堆叠在彼此之上） ）。然后我们通过线性层传递输出（在除去句子长度维度之后）以接收我们的预测。然后我们返回预测，新的隐藏状态和新的单元状态。

In [31]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.dropout = dropout
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        
        self.out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [sent len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #sent len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

Seq2Seq
-------------------------
对于实现的最后部分，我们将实现seq2seq模型。 这将处理：

* 接收输入/源语句
* 使用编码器产生上下文向量
* 使用解码器产生预测的输出/目标句子
* 我们的完整模型将如下所示：

<img src='tmp/img/12.png'>

Seq2Seq模型包含一个编码器，解码器和一个设备（我们稍后会解释）。

对于此实现，我们必须确保编码器和解码器中的层数和隐藏（和单元）维度相等。情况并非总是如此，在序列到序列模型中，您不一定需要相同数量的层或相同的隐藏尺寸。但是，如果您执行的操作具有不同数量的图层，则需要决定如何处理这些图层。例如，如果您的编码器有2层，而您的解码器只有1，那么如何处理？你平均解码器输出的两个上下文向量吗？你是否通过线性层？您是否仅使用最高层的上下文向量？等等。

我们的前向方法采用源句，目标句和教师强制比。在训练我们的模型时使用教师强制比率。解码时，在每个时间步骤，我们将预测目标序列中的下一个标记将来自先前解码的标记。在概率等于教学强制比（teacher_forcing_ratio）的情况下，我们将使用序列中的实际地面实况下一个标记作为下一个时间步骤期间解码器的输入。但是，使用概率1 - teacher_forcing_ratio，我们将使用模型预测的标记作为模型的下一个输入，即使它与序列中的实际下一个标记不匹配。

我们在forward方法中做的第一件事是创建一个输出张量，它将存储我们所有的预测，即$ \text {outputs} = \hat {Y} $。

如果我们应该在GPU上放置模型/张量，设备用于告诉PyTorch。为了确保输出张量在GPU上（如果我们使用的是），我们需要使用.to（self.device）将其发送到设备。

然后，我们将输入/源语句src提供给编码器，并接收最终的隐藏和单元状态。

解码器的第一个输入是序列的开始（`<sos>`）令牌。由于我们的trg张量已经附加了`<sos>`标记（当我们在TRG字段中定义init_token时一直回来），我们通过切入它来获得$ y_1 $。我们知道我们的目标句子应该是多长时间（max_len），所以我们循环多次。在循环的每次迭代期间，我们：

将输入，先前隐藏和前一个单元格状态（$ y_t，s_ {t-1}，c_ {t-1} $）传递给解码器
从解码器接收预测，下一个隐藏状态和下一个单元状态（$ \hat {y} _ {t + 1}，s_ {t}，c_ {t} $）
将我们的预测，$ \hat {y} _ {t + 1} $ /输出放在我们的预测张量中，$ \ hat {Y} $ /输出
决定我们是否要去“教师队伍”
如果我们这样做，下一个输入是序列中的地面实况下一个标记，$ y_ {t + 1} $ / trg [t]
如果我们不这样做，则下一个输入是序列中预测的下一个标记，$ \hat {y} _ {t + 1} $ / top1
一旦我们完成了所有预测，我们就会返回充满预测的张量，$ \hat {Y} $ / outputs。

In [32]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        
        #src = [sent len, batch size]
        #trg = [sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, max_len):
            
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            input = (trg[t] if teacher_force else top1)
        
        return outputs

Training the Seq2Seq Model
---------------------------
现在我们已经实施了模型，我们可以开始训练它。

首先，我们将初始化我们的模型。 如前所述，输入和输出维度由词汇表的大小定义。 编码器和解码器的嵌入尺寸和丢失可以不同，但是层数和隐藏/单元状态的大小必须相同。

然后我们定义编码器，解码器，然后定义我们放置在设备`device`上的Seq2Seq模型。

In [33]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

我们定义了我们的优化器，我们用它来更新训练循环中的参数。

In [34]:
optimizer = optim.Adam(model.parameters())

接下来，我们定义我们的损失函数。`CrossEntropyLoss`函数计算`log softmax`以及我们预测的负对数似然。

这计算每个标记的平均损失，但是通过将`<pad>`标记的索引作为ignore_index参数传递，只要目标标记是填充标记，我们就不会平均丢失。

In [35]:
pad_idx = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

接下来，我们将定义我们的训练循环。

首先，我们将使用`model.train()`将模型设置为“训练模式”。 这将打开`dropout`（和批量规范化，我们没有使用），然后迭代我们的数据迭代器。

在每次迭代时：

* 从批处理中获取源语句和目标语句
*  将从上次迭代计算的梯度归零
* 将源和目标输入模型以获得输出$ \hat{Y} $
* 由于损失函数仅适用于具有1d目标的2d输入，我们需要使用.view来展平它们
  * 我们也不想测量`<sos>`标记的丢失，因此我们切掉了输出的第一列和目标张量用`loss.backward()`计算梯度
* 剪辑渐变以防止它们爆炸（RNN中的常见问题）
* 通过执行优化步骤来更新模型的参数
* 将损失值与运行总计相加

最后，我们返回所有批次的平均损失。

In [36]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [sent len, batch size]
        #output = [sent len, batch size, output dim]
        
        #reshape to:
        #trg = [(sent len - 1) * batch size]
        #output = [(sent len - 1) * batch size, output dim]
        
        loss = criterion(output[1:].view(-1, output.shape[2]), trg[1:].view(-1))
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

我们的评估循环类似于我们的训练循环，但是因为我们没有更新任何参数，所以我们不需要传递优化器或剪辑值。

我们必须记住使用`model.eval()`将模型设置为评估模式。这将关闭丢失（和批量标准化，如果使用）。

我们使用`with torch.no_grad()`块来确保在块内没有计算渐变。这可以减少内存消耗并加快速度。

迭代循环类似（没有参数更新），但是我们必须确保我们让教师强制进行评估。这将导致模型仅使用它自己的预测来在句子中进行进一步的预测，这反映了它在部署中的使用方式。

我们终于可以开始训练我们的模型！

在每个时代，我们将检查我们的模型到目前为止是否已达到最佳验证损失。 如果有，我们将更新我们的最佳验证损失并保存我们模型的参数（在PyTorch中称为`state_dicts`）。 然后，当我们来测试我们的模型时，我们将使用这个最佳验证损失模型。

我们将在每个时代印刷出失落和困惑。 由于数字更大，因此更容易看到困惑的变化而不是损失的变化。

In [37]:
N_EPOCHS = 25
CLIP = 10
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'tut1_model.pt')

best_valid_loss = float('inf')

if not os.path.isdir(f'{SAVE_DIR}'):
    os.makedirs(f'{SAVE_DIR}')

for epoch in range(N_EPOCHS):
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
    
    print(f'| Epoch: {epoch+1:03} | Train Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f} | Val. Loss: {valid_loss:.3f} | Val. PPL: {math.exp(valid_loss):7.3f} |')

NameError: name 'evaluate' is not defined

In [ ]:
model.load_state_dict(torch.load(MODEL_SAVE_PATH))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')